**Log in to CATMAID instance**\
<br>
After importing pymaid and contools, you must log in to your CATMAID instance. You will need to provide your username, password, token, and the url of the CATMAID instance.
Here, these data are stored in pymaid_creds.py and imported into the script. Make sure to add this filename to a .gitignore file to prevent sharing your credentials on github.

In [3]:

import pymaid
import contools

from pymaid_creds import url, name, password, token
rm = pymaid.CatmaidInstance(url, token, name, password)


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


**Signal Cascade Algorithm**\
<br>
Signal cascades simulate multihop signal propagation through a network.
This propagation is probabilistic, but determined by the edge weight (synaptic strength in biological neural networks).

$P(propagation) = 1-(1-p)^n$ , where $n = \sum_{}^{}$ upstream actived synapses
<br>

These propagation calculations are run for each neuron in the network iteratively per hop until the signal has run through the whole network.
Note that when run with $simultaneous=True$, the synaptic strength from multiple upstream neurons is combined to calculate the chance that a downstream neuron is activated.

**Parameter Setup**
- *name*:           
        a string which is the name of the cascade, for logging purposes only

+ *source_skids*:   
        an array [] of CATMAID skeleton IDs, which indicate the starting point(s) of the cascade

- *stop_skids*:     
        an array [] of CATMAID skeleton IDs, which indicate the end point(s) of the cascade. When signal reaches an end point, that portion of the cascade stops

+ *adj*:            
        pandas DataFrame of the adjacency matrix of the network, with skeleton IDs for row and column names

- *p*:              
        probability of propagating signal per individual synapse, suggested value = 0.05

+ *max_hops*:       
        the max number of hops the cascade will run

- *n_init*:         
        the number of iterations. because cascades are probabilistic, multiple cascades will be run and their results summed

+ *simultaneous*:   
        whether neurons that are simultaneously active can sum their outputs to more effectively activate their shared downstream partners


In [ ]:
name = 'CN-4 cascade' 
source_skids = pymaid.get_skids_by_annotation('CN-4') 
stop_skids = pymaid.get_skids_by_annotation('mw dVNC') 
adj = contools.Promat.pull_adj('ad', 'brain and accessory') # can also load and use a CSV instead
p = 0.05
max_hops = 10
n_init = 100
simultaneous = True


**Running the Cascade**\
<br>
Assemble the previously defined variables and input into the appropriate function contools.Cascade_Analyzer.run_single_cascade()

In [ ]:

cascade = contools.Cascade_Analyzer.run_single_cascade(
    name = name,
    source_skids = source_skids,
    stop_skids = stop_skids,
    adj = adj,
    p = p,
    max_hops = max_hops,
    n_init = n_init,
    simultaneous = simultaneous
)

cascade